Задача: есть 2 набора стихов-"пирожков", которые мы хотим научиться отличать друг от друга.

## 0. Подготовка данных

In [1]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from functools import reduce

In [2]:
def read_data(filename):
    buf = []
    with open(filename) as f:
        for line in f:
            l = line.strip()
            if len(l) > 0:
                buf.append(l)
            else:
                if len(buf) > 0:
                    yield buf
                    buf = []
        if len(buf) > 0:
            yield buf

class0 = list(read_data("test_data/class0.txt"))
class1 =  list(read_data("test_data/class1.txt"))

Посмотрим, как выглядят данные:

In [3]:
class0[:2]

[['ребята мне совсем хреново',
  'друзьям аркадий говорит',
  'друзья качают головами',
  'и тычут палочкой в нево'],
 ['придя домой семён наткнулся',
  'на лужу серной кислоты',
  'скорее вытирать скорее',
  'пока к соседям не прожгло']]

In [4]:
class1[:2]

[['дантес сойду еще не речи',
  'потенциен от укла нет',
  'была ушли в уборной шляпе',
  'высоковольтный педикюр',
  'быстрят особенное снежно',
  'с вилкой уже сказал блины'],
 ['я понял то что не увозит',
  'и чо опасниками день',
  'а он настроит рыбин к людек',
  'а над оконною женой',
  'что ж кто мечтательно подходит',
  'и был любимой словно тонн']]

И там, и там стихи многострочные. Обычно пирожки содержат 4 строки. Мы можем посмотреть, сколькистрочные стихи содержат оба класса и выкинуть то, что 4 строки не содержит - по определению, это не "пирожок".
Проверим количество строк в классах:

In [5]:
from collections import Counter
print("Проверка количества строк в пирожках")
print("Класс 0: %s" % Counter(map(len, class0)))
print("Класс 1: %s" % Counter(map(len, class1)))

Проверка количества строк в пирожках
Класс 0: Counter({4: 1095})
Класс 1: Counter({4: 785, 2: 110, 6: 87, 8: 19, 3: 4, 10: 3, 1: 3, 14: 1})


В классе 1 есть довольно много стихов не из 4 строк. Уберем их из рассмотрения и случайным образом продублируем стихи из класса 1, чтобы сбалансировать количество стихов в каждом из классов.

In [7]:
class1 = list(filter(lambda x: len(x)==4, class1))

class1 += list(
    map(lambda i: class1[i], 
        np.random.randint(0, len(class1), size=len(class0)-len(class1))))

len(class0), len(class1), Counter(list(map(len, class0))), Counter(list(map(len, class1)))

(1095, 1095, Counter({4: 1095}), Counter({4: 1095}))

Теперь все хорошо. Выделим обучающую и тестовую выборки, чтобы дальше работать с ними.

In [8]:
x_data = list(map(lambda x:"\n".join(x), class0 + class1))
y_data = np.concatenate((np.zeros(len(class0)),np.ones(len(class1))), axis=0)  

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, stratify=y_data)

### Общие выводы

Пирожки из класса 1 пытаются мимикрировать под пирожки из класса 0, в которых часто есть ошибки и опечатки. Для класса 1 характерны не естественные для человека опечатки, новые слова, не связанность текста. Каждое стихотворение из класса 0 семантически согласовано - все строки связаны либо по смыслу, либо грамматически. Человек не будет употреблять глагол вместо прилагательного. В классе 1 такие примеры есть и они бросаются в глаза. 

Можно сравнить:

1. структуру текста (в том числе используя представление о структуре пирожков)

2. используя какую-либо метрику, например, расстояние Левенштейна, определять опечатки. Пытаться определить смысл оригинальных слов и посмотреть, насколько это будет влиять на точность определения структуры текста.

3. определить основные части речи (существительное, глагол) в тексте и сравнить, насколько они употребимы вместе друг с другом.

4. посмотреть прилагательные

## 1a. bag-of-words (простой)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier

count_vectorizer = CountVectorizer().fit(x_train)

x_train_vect = count_vectorizer.transform(x_train)
x_test_vect = count_vectorizer.transform(x_test)

In [10]:
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(lr, param_grid, cv=10)
grid.fit(x_train_vect, y_train)
print("Лучший score на обучающей выборке: ", grid.best_score_)

print("Score лучшей модели на тестовой выборке:", 
      np.mean(
          np.equal(
              grid.best_estimator_.predict(x_test_vect), 
              y_test)))

Лучший score на обучающей выборке:  0.654689403167
Score лучшей модели на тестовой выборке: 0.655109489051


Попробуем использовать TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(TfidfVectorizer(min_df=5, norm=None), LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(pipe, param_grid, cv=10)
grid.fit(x_train, y_train)
print("Лучший score на обучающей выборке: ", grid.best_score_)
print("Score лучшей модели на тестовой выборке",
      np.mean(
          np.equal(
              grid.best_estimator_.predict(x_test), 
              y_test)))

Лучший score на обучающей выборке:  0.601096224117
Score лучшей модели на тестовой выборке 0.569343065693


In [12]:
# ну и дерево - для сравнения
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier().fit(x_train_vect, y_train)
print("Score на обучающей выборке: ", np.mean(
          np.equal(
              dt.predict(x_train_vect), 
              y_train)))

print("Score на тестовой выборке:", 
      np.mean(
          np.equal(
              dt.predict(x_test_vect), 
              y_test)))

Score на обучающей выборке:  1.0
Score на тестовой выборке: 0.647810218978


Вывод: bag of words без доп.обработки текстов дает не очень хороший результат. Это может быть связано с тем, что в обоих классах присутствуют несловарные слова (слова с опечатками), в классе 0 - имена собственные (Путин, Куклачев и др.), много уникальных слов (дерево отличает чуть лучше - видимо, дело в том, что все-таки эти несловарные слова позволяют отличать класс 1 от класса 0 чуть лучше, хотя и ненамного).

## 1b.  bag-of-words с предобработкой

In [13]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [14]:
import nltk 

from nltk.corpus import stopwords
ru_stopwords = set(stopwords.words('russian'))

Попробуем добавить простой метод для разбивки пирожков на слова, который будет приводить их к нормальной форме и сравним score логистической регрессии для разных вариантов обработки.

In [15]:
def make_custom_tokenizer(stopwords={}):
    def _custom_tokenizer(s):
        xs = s.split()
        return list(filter(lambda x: x not in stopwords,
                      map(lambda word: morph.parse(word)[0].normal_form, xs)))
    return _custom_tokenizer

custom_tokenizer = make_custom_tokenizer()
custom_tokenizer("А роза упала на лапу Азора")

['а', 'роза', 'упасть', 'на', 'лапа', 'азор']

In [16]:
custom_tokenizer2 = make_custom_tokenizer(ru_stopwords)

In [17]:
from sklearn.model_selection import cross_val_score
x_train_vect = CountVectorizer(
        max_features=10000, 
        max_df=.15, 
        tokenizer=custom_tokenizer
    ).fit(x_train).transform(x_train)

scores = cross_val_score(LogisticRegression(), x_train_vect, y_train, cv=10)
print("Score на обучающей выборке без выкидывания стоп-слов", np.mean(scores))


vect = CountVectorizer(
        max_features=10000, 
        max_df=.15, 
        tokenizer=custom_tokenizer2
    ).fit(x_train)
x_train_vect = vect.transform(x_train)

scores = cross_val_score(LogisticRegression(), x_train_vect, y_train, cv=10)
print("С выкидыванием стоп-слов", np.mean(scores))

Score на обучающей выборке без выкидывания стоп-слов 0.63394798707
С выкидыванием стоп-слов 0.629672347928


### 2. Простая модель с word2vec и здравым смыслом

Если посмотреть на данные обоих классов, то заметны следующие отличия:

1. В классе 0 слова в отдельных строках связаны между собой, в классе 1 часто не связаны никак
2. В классе 1 встречаются несогласованные по родам и временам пары "прилагательное-существительное", "существительное-глагол", в то время как в классе 0 их нет (вероятно, потому, что как правило, человек такие ошибки не допускает)

Можно попробовать: 
1. взять обученную word2vec-модель, с ее помощью попарно сравнить строки, найти в них пары наиболее похожих слов, соответствующие числовые характеристики использовать в качестве признаков.
2. добавить булевые признаки - наличие несоответствий в строках

In [18]:
import gensim
from gensim.models.keyedvectors import KeyedVectors
keyed_vect = KeyedVectors.load_word2vec_format("ruscorpora_1_300_10.bin.gz", binary=True, encoding='utf-8')

In [21]:
# в ruscorpora используется отличный от pymorphy2 формат POS-тэгов, слова в модели идут уже с ними.
# Следующий метод получает нужный тег и приписывает его к слову, сохраняя его оригинальную форму.
def preprocess_word(x):
    forms = morph.parse(x)
    for m in forms:
        if m.tag.POS is None:
            #print(x, m.tag.POS)
            continue
        if m.tag.POS.startswith('ADJ'):
            return m.normal_form + "_ADJ"
        if m.tag.POS =='INFN':
            return m.normal_form + "_VERB"
        if m.tag.POS == 'NPRO':
            return m.normal_form + "_PRON"
        return m.normal_form + "_" + m.tag.POS
    return x + "_UNKNOWN"

preprocess_word("двойка") 

'двойка_NOUN'

In [22]:
def get_similarity(line1, line2):
    line1 = [preprocess_word(x) for x in line1.split() if not x in ru_stopwords]
    line2 = [preprocess_word(x) for x in line2.split() if not x in ru_stopwords]
    m = 0
    i0=-1
    j0=-1
    for i in line1:
        for j in line2:
            if not i.split('_')[1] in set(['NOUN', 'VERB', 'ADJ', 'PRON']):
                continue
            if not j.split('_')[1] in set(['NOUN', 'VERB', 'ADJ','PRON']):
                continue
            if not i in keyed_vect.vocab:
                i1 = morph.parse(i.split("_")[0])[0]
                j1 = morph.parse(j.split("_")[0])[0]
                if i1.normal_form == j1.normal_form:
                    return i, j, 1.0
                continue
            if not j in keyed_vect.vocab:
                i1 = morph.parse(i.split("_")[0])[0]
                j1 = morph.parse(j.split("_")[0])[0]
                if i1.normal_form == j1.normal_form:
                    return i, j, 1.0
                continue
            if keyed_vect.similarity(i, j) > m:
                m=keyed_vect.similarity(i, j)
                i0=i
                j0=j
    return i0, j0, m


get_similarity("а роза упала на лапу азора", "а зори здесь тихие")

('роза_NOUN', 'зоря_NOUN', 0.24094977034332399)

In [23]:
def preprocess_poem(poem):
    if isinstance(poem, str):
        poem = poem.split('\n')
    for i in range(4):
        for j in range(i + 1,4):
            yield i, j, get_similarity(poem[i], poem[j])

In [24]:
list(preprocess_poem(x_data[0])), print(x_data[0])

ребята мне совсем хреново
друзьям аркадий говорит
друзья качают головами
и тычут палочкой в нево


([(0, 1, ('ребята_NOUN', 'друг_NOUN', 0.3061908933626224)),
  (0, 2, ('ребята_NOUN', 'друг_NOUN', 0.3061908933626224)),
  (0, 3, ('ребята_NOUN', 'тыкать_VERB', 0.24149795902845334)),
  (1, 2, ('друг_NOUN', 'друг_NOUN', 1.0000000000000004)),
  (1, 3, ('говорить_VERB', 'тыкать_VERB', 0.33467823884290354)),
  (2, 3, ('качать_VERB', 'тыкать_VERB', 0.37435990903307714))],
 None)

In [29]:
# добавляет попарные значения похожести в качестве признаков, а также, отдельным признаком - 
# максимальное значение похожести среди всех пар строк
def processed_to_features(xs):
    x = [np.concatenate((np.asarray([k[-1] for i, j, k in (preprocess_poem(x))
                    ]).T, np.asarray([
        max(map(lambda x: x[2][-1], preprocess_poem(x)))]))) for x in xs]
    return np.vstack(x)


In [35]:
custom_features = processed_to_features(x_train)

In [36]:
custom_features.shape

(1642, 7)

In [37]:
def check_line(line):
    "проверяет строку на наличие проблем согласования родов"
    xs = [x for x in line.split()]
    for i in range(len(xs)-1):
        x1 = morph.parse(xs[i])[0]
        x2 = morph.parse(xs[i + 1])[0]
        if x1.tag.POS is None:
            continue
        if x2.tag.POS is None:
            continue
        for x1, x2 in [(x1, x2), (x2, x1)]:
            if x1.tag.POS.startswith('ADJF') and x2.tag.POS=='NOUN':
                if x1.tag.number != x2.tag.number or x1.tag.gender != x2.tag.gender:
                    #print(x1, x2)
                    return True
        if x1.tag.POS.startswith('NOUN') and x2.tag.POS=='VERB':
            if x1.tag.number != x2.tag.number or (x1.tag.gender != x2.tag.gender and "impf" not in x2.tag):
                #print(x1, x2)
                return True
    return False
                
def check_pairs(poem):
    """This tries to check nouns in lines and returns true/false values (or counts?) based on 
    how many cases there were with incorrect part of speech
    """
    x = [check_line(line) for line in poem.split('\n')] 
    return x

check_line('а зеленая роза упал на лапу азора')

In [38]:
boolean_features = np.vstack([np.asarray(check_pairs(x)) for x in x_train])

In [39]:
X_features = np.concatenate((custom_features, boolean_features), axis=1)

In [40]:
X_features

array([[ 0.27090745,  0.30467402,  0.28084496, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.35168789,  0.        ,  0.08408675, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.34805499,  0.22577519,  0.4770446 , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.16354329,  0.23429986,  0.27524324, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.32957714,  0.5960909 ,  0.26527108, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.20761791,  0.        ,  0.28771346, ...,  1.        ,
         0.        ,  0.        ]])

In [51]:
scores = cross_val_score(LogisticRegression(), X_features, y_train, cv=10)
print("Средний score кросс-валидации на обучающей выборке", np.mean(scores))

Средний score кросс-валидации на обучающей выборке 0.737525712607


In [42]:
custom_features = processed_to_features(x_test)

In [45]:
boolean_features = np.vstack([np.asarray(check_pairs(x)) for x in x_test])

In [46]:
X_features_test = np.concatenate((custom_features, boolean_features), axis=1)

In [49]:
lr = LogisticRegression().fit(X_features, y_train)
print("Score логистической регрессии на тестовой выборке", 
      np.mean(np.equal(lr.predict(X_features_test), y_test)))

Score логистической регрессии на тестовой выборке 0.724452554745


### Выводы
Как видим, даже совсем простой анализ текста приводит к небольшому, но заметному улучшению результата по сравнению с bag-of-words.